# Exemplary SA-CCR allocation under consideration of an initial margin threshold

We consider a portfolio of two trades, an IRS and an equity option.

In [1]:
import QuantLib as ql
from scipy import optimize
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, InitialMargining
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import export
from margining.noimm import NOIMM
from marketdata.interestRateIndices import InterestRateIndex
from marketdata import init_marketdata
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import Stock, TradeType, TradeDirection
asdf = 1


In [2]:
irs = IRS(notional=100000000,
          index = InterestRateIndex.USDLIBOR3M,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years))

ca_irs = CollateralAgreement(initialMargining=InitialMargining.NO_IM)
ca_irs.link_sa_ccr_instance(SA_CCR(ca_irs))
ca_irs.add_trades(irs)

In [3]:
def find_notional_for_eq_ead(notional):
   eqOpt = EquityOption(notional = notional)
   global ca_irs
   ref_ead = ca_irs.get_sa_ccr_model().get_risk_measure()
   trades = ca_irs.trades
   ca_irs.remove_all_trades()
   ca_irs.add_trades(eqOpt)
   compare_ead = ca_irs.get_sa_ccr_model().get_risk_measure()
   ca_irs.remove_all_trades()
   ca_irs.add_trades(trades)
   return (ref_ead - compare_ead)**2

eqOptNot = optimize.minimize(find_notional_for_eq_ead, x0=10000)

In [4]:
eqOpt = EquityOption(notional = 403690.28,
                     maturity = ql.Period(1, ql.Years),
                     underlying = Stock.ADS,
                     tradeType = TradeType.CALL,
                     tradeDirection = TradeDirection.LONG)

In [5]:
ca_eqOpt = CollateralAgreement(initialMargining=InitialMargining.NO_IM)
ca_eqOpt.link_sa_ccr_instance(SA_CCR(ca_eqOpt))
ca_eqOpt.add_trades(eqOpt)

These two trades happen to have the same EAD under VM only


In [6]:
f'IRS EAD under VM: {ca_irs.get_sa_ccr_model().get_ead():,.0f} USD'

'IRS EAD under VM: 1,651,405 USD'

In [7]:
f'EQ EAD under VM: {ca_eqOpt.get_sa_ccr_model().get_ead():,.0f} USD'

'EQ EAD under VM: 1,651,405 USD'

If they are put into a common portfolio under VM only this common portfolio just has the sum as EAD as no hedge effects take place.

In [8]:
ca_common = CollateralAgreement(
    initialMargining=InitialMargining.NO_IM)
ca_common.link_sa_ccr_instance(SA_CCR(ca_common))
ca_common.add_trades([eqOpt,irs])

In [9]:
f'Portfolio EAD under VM: {ca_common.get_sa_ccr_model().get_ead():,.0f} USD'

'Portfolio EAD under VM: 3,302,809 USD'

However the IM and consequently the EAD under IM differ between the two trades.

In [10]:
ca_eqOpt.initialMargining = InitialMargining.SIMM
ca_irs.initialMargining = InitialMargining.SIMM

In [11]:
f'EQ IM: {ca_eqOpt.get_im_model().get_risk_measure():,.0f} USD'

'EQ IM: 2,806,744 USD'

In [12]:
f'IRS IM: {ca_irs.get_im_model().get_risk_measure():,.0f} USD'

'IRS IM: 4,519,079 USD'

In [13]:
f'EQ EAD under VM and IM: {ca_eqOpt.get_sa_ccr_model().get_ead():,.0f} USD'

'EQ EAD under VM and IM: 530,996 USD'

In [14]:
f'IRS EAD under VM and IM: {ca_irs.get_sa_ccr_model().get_ead():,.0f} USD'

'IRS EAD under VM and IM: 291,441 USD'

The IM of the common portfolio is the sum of the individual portfolios while the EAD of the common portfolio is a bit smaller than the respective sum.

In [15]:
ca_common.initialMargining = InitialMargining.SIMM

In [16]:
f'Portfolio IM: {ca_common.get_im_model().get_risk_measure():,.0f} USD'

'Portfolio IM: 7,325,823 USD'

In [17]:
f'Portfolio EAD under VM and IM: {ca_common.get_sa_ccr_model().get_ead():,.0f} USD'

'Portfolio EAD under VM and IM: 777,229 USD'

We can compare the allocation of EAD with and without IM to see that with IM, less is attributed  to the IRS as it commands a higher margin.

In [18]:
eulerAllocator = EulerAllocator(ca_common)
eulerAllocator.fdApproach2=FdApproach2.Central
ca_common.initialMargining = InitialMargining.NO_IM
{k.__class__.__name__: f'{v:,.0f} USD'
 for k, v in eulerAllocator.allocate_ead().items()}

{'EquityOption': '1,651,405 USD', 'IRS': '1,651,405 USD'}

In [19]:
ca_common.initialMargining = InitialMargining.SIMM
{k.__class__.__name__: f'{v:,.0f} USD'
 for k, v in eulerAllocator.allocate_ead().items()}

{'EquityOption': '505,528 USD', 'IRS': '271,701 USD'}

We introduce an initial margin threshold that is higher than the calculated IM and see that the resulting allocation is subadditive and the same as in the case without initial margin.

In [20]:
ca_common.threshold = 10000000
{k.__class__.__name__: f'{v:,.0f} USD'
 for k, v in eulerAllocator.allocate_ead().items()}

{'EquityOption': '1,651,405 USD', 'IRS': '1,651,405 USD'}

In [21]:
f'Portfolio EAD with 10Mn IM threshold: {ca_common.get_sa_ccr_model().get_ead():,.0f} USD'

'Portfolio EAD with 10Mn IM threshold: 3,302,809 USD'

When we lower the threshold below the exchanged IM the allocation fails

In [22]:
ca_common.threshold = 5000000
{k.__class__.__name__: f'{v:,.0f} USD'
 for k, v in eulerAllocator.allocate_ead().items()}

{'EquityOption': '331,448 USD', 'IRS': '-381,962 USD'}

In [23]:
f'Portfolio EAD with 10Mn IM threshold: {ca_common.get_sa_ccr_model().get_ead():,.0f} USD'

'Portfolio EAD with 10Mn IM threshold: 2,032,638 USD'

Further discussion of these results in section \ref{sec:Incorporation of an IM threshold}.

In [25]:
export("Exemplary SA-CCR allocation initial margin threshold 2.ipynb")